In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


('1.10.0', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'2j4y3my2'

In [3]:
wandb.init(id='2rrh11ch', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet(model_arch="efficientnet_b3").to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

14029.957

In [5]:
m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m_9.21_e2.torch"), strict=False)

<All keys matched successfully>

In [6]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=5e-5)

In [7]:
%%time
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

CPU times: user 1min 4s, sys: 2.33 s, total: 1min 6s
Wall time: 1min 6s


In [8]:
%%time
#rw_dataloader = RealWorldDataloader()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


In [9]:
"""for n, p in m.named_parameters():
    if 'fcs_2' in n:
        p.requires_grad = True
    else:
        p.requires_grad = False
        
    print(n, p.requires_grad)"""

"for n, p in m.named_parameters():\n    if 'fcs_2' in n:\n        p.requires_grad = True\n    else:\n        p.requires_grad = False\n        \n    print(n, p.requires_grad)"

In [10]:
model_stem = "9.21"
START_E = 0

In [ ]:
n_epochs = 10000
log_wandb = True

for e in range(START_E, n_epochs):
    
    opt.param_groups[0]['lr'] = 6e-4 if e < 40 else 3e-4 if e < 80 else 1e-4

    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=1280)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

EPOCH 0

{'trn_control_loss': 0.14795351, 'trn_headings_loss': 0.00665626, 'trn_curvatures_loss': 0.01345843, 'consistency losses/trn_steer_cost': 0.01410523, 'consistency losses/trn_te_loss': 24.49856246, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9921875, 'consistency losses/trn_torque_loss': 3960.70851326, 'consistency losses/trn_torque_delta_loss': 54.23098539, 'logistical/obs_consumed_per_second': 198.71653543, 'logistical/obs_generated_per_second': 165.77165354, 'logistical/data_consumption_ratio': 1.19963421, 'logistical/manual_train_pause': 0.0, 'logistical/max_param': 16.28517151, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 192.609375, 'logistical/obs_generated_per_second': 166.8359375, 'logistical/data_consumption_ratio': 1.15547901, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.17503405, 'trn_headings_loss': 0.00823984, 'trn_curvatures_loss': 0.01932466, 'consistency losses/trn_steer_cost': 0.02019936, 'consistency losses/trn_

{'logistical/obs_consumed_per_second': 197.1640625, 'logistical/obs_generated_per_second': 167.59375, 'logistical/data_consumption_ratio': 1.17685402, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.21421146, 'trn_headings_loss': 0.00982752, 'trn_curvatures_loss': 0.03545535, 'consistency losses/trn_steer_cost': 0.02226922, 'consistency losses/trn_te_loss': 22.86142823, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9921875, 'consistency losses/trn_torque_loss': 3853.92084789, 'consistency losses/trn_torque_delta_loss': 50.66002306, 'logistical/max_param': 16.29849434, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 198.8046875, 'logistical/obs_generated_per_second': 170.46875, 'logistical/data_consumption_ratio': 1.16660015, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.16901588, 'trn_headings_loss': 0.00674386, 'trn_curvatures_loss': 0.01331535, 'consistency losses/trn_steer_cost': 0.01541898, 'consistency losses/trn_te_loss': 22.7

{'logistical/obs_consumed_per_second': 198.5703125, 'logistical/obs_generated_per_second': 169.625, 'logistical/data_consumption_ratio': 1.17126016, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.25913763, 'trn_headings_loss': 0.01158294, 'trn_curvatures_loss': 0.02503312, 'consistency losses/trn_steer_cost': 0.01949763, 'consistency losses/trn_te_loss': 21.90722102, 'consistency losses/trn_%_updates_w_torque_delta_loss': 1.0, 'consistency losses/trn_torque_loss': 3860.75404787, 'consistency losses/trn_torque_delta_loss': 63.85776734, 'logistical/max_param': 16.29040718, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 198.6328125, 'logistical/obs_generated_per_second': 168.2734375, 'logistical/data_consumption_ratio': 1.18071313, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.12690043, 'trn_headings_loss': 0.00565639, 'trn_curvatures_loss': 0.01168126, 'consistency losses/trn_steer_cost': 0.01019013, 'consistency losses/trn_te_loss': 23.4319876

{'logistical/obs_consumed_per_second': 198.03125, 'logistical/obs_generated_per_second': 167.578125, 'logistical/data_consumption_ratio': 1.18166461, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.11152482, 'trn_headings_loss': 0.00500835, 'trn_curvatures_loss': 0.01116699, 'consistency losses/trn_steer_cost': 0.01133052, 'consistency losses/trn_te_loss': 23.48823296, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9921875, 'consistency losses/trn_torque_loss': 4237.66248322, 'consistency losses/trn_torque_delta_loss': 42.24161581, 'logistical/max_param': 16.29615784, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 129.0, 'logistical/obs_generated_per_second': 169.0, 'logistical/data_consumption_ratio': 0.76672981, 'logistical/manual_train_pause': 0.0}


 Eval...
EPOCH 4

{'trn_control_loss': 0.10571051, 'trn_headings_loss': 0.005165, 'trn_curvatures_loss': 0.01121134, 'consistency losses/trn_steer_cost': 0.01190495, 'consistency losses/trn_te_los

{'logistical/obs_consumed_per_second': 198.7578125, 'logistical/obs_generated_per_second': 166.765625, 'logistical/data_consumption_ratio': 1.19296464, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.06841493, 'trn_headings_loss': 0.00376201, 'trn_curvatures_loss': 0.00878257, 'consistency losses/trn_steer_cost': 0.00810894, 'consistency losses/trn_te_loss': 22.78220779, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9921875, 'consistency losses/trn_torque_loss': 3950.84282875, 'consistency losses/trn_torque_delta_loss': 29.41000394, 'logistical/max_param': 16.28732872, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 199.8671875, 'logistical/obs_generated_per_second': 163.125, 'logistical/data_consumption_ratio': 1.22521235, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.06187093, 'trn_headings_loss': 0.00319275, 'trn_curvatures_loss': 0.00853427, 'consistency losses/trn_steer_cost': 0.00971758, 'consistency losses/trn_te_loss': 22.57

...

{'logistical/obs_consumed_per_second': 198.1875, 'logistical/obs_generated_per_second': 167.2578125, 'logistical/data_consumption_ratio': 1.18530645, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.07527471, 'trn_headings_loss': 0.00336333, 'trn_curvatures_loss': 0.00754613, 'consistency losses/trn_steer_cost': 0.00890881, 'consistency losses/trn_te_loss': 23.00024603, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.984375, 'consistency losses/trn_torque_loss': 4414.44416237, 'consistency losses/trn_torque_delta_loss': 42.77784028, 'logistical/max_param': 16.28543282, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 197.8984375, 'logistical/obs_generated_per_second': 167.7734375, 'logistical/data_consumption_ratio': 1.17996064, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.05354202, 'trn_headings_loss': 0.00241403, 'trn_curvatures_loss': 0.00598934, 'consistency losses/trn_steer_cost': 0.00798596, 'consistency losses/trn_te_loss': 24.0

{'logistical/obs_consumed_per_second': 199.8046875, 'logistical/obs_generated_per_second': 167.8671875, 'logistical/data_consumption_ratio': 1.19050481, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.07733464, 'trn_headings_loss': 0.00334079, 'trn_curvatures_loss': 0.00820412, 'consistency losses/trn_steer_cost': 0.0115765, 'consistency losses/trn_te_loss': 23.39716342, 'consistency losses/trn_%_updates_w_torque_delta_loss': 1.0, 'consistency losses/trn_torque_loss': 4364.18379021, 'consistency losses/trn_torque_delta_loss': 44.89715799, 'logistical/max_param': 16.28225708, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 197.8203125, 'logistical/obs_generated_per_second': 168.328125, 'logistical/data_consumption_ratio': 1.17541758, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.15276659, 'trn_headings_loss': 0.00670008, 'trn_curvatures_loss': 0.01836692, 'consistency losses/trn_steer_cost': 0.01385789, 'consistency losses/trn_te_loss': 22.42717

{'logistical/obs_consumed_per_second': 193.0078125, 'logistical/obs_generated_per_second': 164.484375, 'logistical/data_consumption_ratio': 1.17424421, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.03708303, 'trn_headings_loss': 0.00176865, 'trn_curvatures_loss': 0.00359835, 'consistency losses/trn_steer_cost': 0.00660945, 'consistency losses/trn_te_loss': 23.5592092, 'consistency losses/trn_%_updates_w_torque_delta_loss': 1.0, 'consistency losses/trn_torque_loss': 4489.40486383, 'consistency losses/trn_torque_delta_loss': 32.07080705, 'logistical/max_param': 16.26301765, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 201.3125, 'logistical/obs_generated_per_second': 159.109375, 'logistical/data_consumption_ratio': 1.26680136, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.05404806, 'trn_headings_loss': 0.00247475, 'trn_curvatures_loss': 0.00691698, 'consistency losses/trn_steer_cost': 0.00988097, 'consistency losses/trn_te_loss': 23.15948415,

{'logistical/obs_consumed_per_second': 203.2578125, 'logistical/obs_generated_per_second': 167.96875, 'logistical/data_consumption_ratio': 1.21215326, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.06973815, 'trn_headings_loss': 0.00280657, 'trn_curvatures_loss': 0.00597595, 'consistency losses/trn_steer_cost': 0.01025024, 'consistency losses/trn_te_loss': 23.74861714, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9921875, 'consistency losses/trn_torque_loss': 3745.08042336, 'consistency losses/trn_torque_delta_loss': 34.98221802, 'logistical/max_param': 16.25979996, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 196.5234375, 'logistical/obs_generated_per_second': 157.484375, 'logistical/data_consumption_ratio': 1.24834412, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.05719173, 'trn_headings_loss': 0.00296824, 'trn_curvatures_loss': 0.00697246, 'consistency losses/trn_steer_cost': 0.00894901, 'consistency losses/trn_te_loss': 22.

{'logistical/obs_consumed_per_second': 197.7265625, 'logistical/obs_generated_per_second': 164.0859375, 'logistical/data_consumption_ratio': 1.20602745, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.03305894, 'trn_headings_loss': 0.00151282, 'trn_curvatures_loss': 0.00429921, 'consistency losses/trn_steer_cost': 0.00682135, 'consistency losses/trn_te_loss': 22.68192172, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9765625, 'consistency losses/trn_torque_loss': 4228.08503342, 'consistency losses/trn_torque_delta_loss': 23.11069001, 'logistical/max_param': 16.26397705, 'logistical/lr': 0.0006}
{'logistical/obs_consumed_per_second': 202.25, 'logistical/obs_generated_per_second': 166.8515625, 'logistical/data_consumption_ratio': 1.21276505, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.04042435, 'trn_headings_loss': 0.00198549, 'trn_curvatures_loss': 0.00568537, 'consistency losses/trn_steer_cost': 0.00866951, 'consistency losses/trn_te_loss': 21.99

In [12]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e58.torch")

Error in callback <function _WandbInit._resume_backend at 0x7fc994177ee0> (for pre_run_cell):


Exception: The wandb backend process has shutdown

Error in callback <function _WandbInit._pause_backend at 0x7fc95a4540d0> (for post_run_cell):


Exception: The wandb backend process has shutdown

In [3]:
set_should_stop(True)

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
